# Depth Cost

In this notebook, we will check if our assumed depth count model, denoted as $\textsf{depth}(\cdot)$, is valid. The depth is derived as moment length over a quantum circuit when decomposed into $CNOT$ and single-qubit gates. For simplicity, we did not assume single-qubit gate merging between the moments of the circuit. The depth costs of the following quantum circuits are included :  
- Draper Adder 
- Doubly Controlled-Draper Adder
- Grover Iterate.


### Check Unitary Equivalence 

The cost derivation is done by decomposing quantum circuits into CNOT and single-qubit gates. The decomposition rules are known ones, but to ensure correctness, we added an equivalence check between the original quantum circuit and the decomposed one, which involves unitary matrix calculations. This requires a lot of computation power for larger qubit size (n). To bypass the equivalence check, comment out the code `cirq.allclose_up_to_global_phase`.

## Draper Adder

The decomposition of Draper Adder is done by decomposing QFT and (single-)controlled phases.
Firstly, Drapper Adder is implemented via $CR_k$ gates and $QFT$, then each gate is decomposed into $CNOT$+single qubit gates by defined rule which is implemented in `./casestudy_pgm/draper_related_utils.py`. Check that custom gate class `my_QFT` and `my_C_R_k_gate`.


In [1]:
from sympy import symbols
import cirq
from casestudy_pgm.draper_adder.draper_correct import draper_true, my_draper_true
from qruntest.symbolic_cost_model import sym_moment_cost_for_my_draper
from qruntest.utils import keep
num_qbit_for_pgm = symbols("n") 
depth_cost_fmla = sym_moment_cost_for_my_draper()
print(f"Depth of Draper Adder : {depth_cost_fmla}")
for n in range(4,10,2):  # since classic simulation(=unitary simulation) in equivalence check between original one and decomposed on may expensive ,
                         #  only try on small qubits
                         # if not to check 
    print(f"Target Program Qubit Len : {n}")
    m = int(n/2)
    ref_qc  = draper_true(m)
    decomposed = cirq.Circuit([x for x in cirq.decompose(my_draper_true(m = m ), keep=keep)])
    assert cirq.allclose_up_to_global_phase(cirq.unitary(ref_qc), cirq.unitary(decomposed))
    depth_cost_fmla = sym_moment_cost_for_my_draper()
    assert len(decomposed) == depth_cost_fmla.subs({num_qbit_for_pgm : n})
    print("Decomposed circuit")
    print(decomposed)
print("Assertion Passed")

Depth of Draper Adder : 10*n - 20
Target Program Qubit Len : 4
Decomposed circuit
                                                            ┌─────┐
0: ──────────────────────────────@───────────────────────────────────────────────────────────────────
                                 │
1: ───T──────────────────────────┼───────────@──────────@────@───────────────────────────────────────
                                 │           │          │    │
2: ───H───T───X───T^-1───X───H───X───H───T───X───T^-1───X────┼T^-1───────────────────X───T───X───H───
              │          │                                   │                       │       │
3: ───T───────@──────────@───H───H───────────────────────────X────────H───H───T^-1───@───────@───────
                                                            └─────┘
Target Program Qubit Len : 6
Decomposed circuit
                                             ┌──┐                                                    ┌────────┐                      

## Controlled Draper Adder

The decomposition of Draper Adder is done by decomposing QFT and (double-)controlled phases. Firstly, Drapper Adder is implemented via $CCCR_k$ gates and $QFT$, then each gate is decomposed into $CNOT$+single qubit gates by defined rule which is implemented in `./casestudy_pgm/draper_related_utils.py`. Check that custom gate class `my_QFT` and `my_triple_C_R_k_gate`.

In [ ]:
from sympy import symbols
import cirq
from casestudy_pgm.double_con_draper_adder.double_con_draper_correct import double_con_draper_true, my_double_con_draper_true
from qruntest.symbolic_cost_model import sym_moment_cost_for_my_double_con_draper
from qruntest.utils import keep
num_qbit_for_pgm = symbols("n") 
depth_cost_fmla = sym_moment_cost_for_my_double_con_draper()

print(f"Depth of Draper Adder : {depth_cost_fmla}")
for n in range(6,12,2):  # since classic simulation(=unitary simulation) may expensive, only try on small qubits
    print(f"Target Program Qubit Len : {n}")
    m = int((n-2)/2)
    ref_qc  = double_con_draper_true(m)
    decomposed = cirq.Circuit([x for x in cirq.decompose(my_double_con_draper_true(m = m), keep=keep)])
    assert cirq.allclose_up_to_global_phase(cirq.unitary(ref_qc), cirq.unitary(decomposed))
    assert len(decomposed) == depth_cost_fmla.subs({num_qbit_for_pgm : n})
print("Assertion Passed")


## Grover Iterate (denoted as $G$ in the paper)

The decomposition of Draper Adder is done by decomposing multiple anti-controlled gates $NC^mX$.
Firstly, $NC^mX$ is decomposed into $NCNCX$, and then $NCNCX$ is further decomposed into $X$ and $CNOT$. The rules are defined in `./qruntest/qc_utils.py` within the custom gate class `my_antianti_con_toff`.

Note that the depth cost of $G$ is a function of $n$, which is the qubit size of the target program to test, NOT the qubit size of $G$.
When $G$ is applied for testing an $n$-qubit target prgoram (as part of running $s$-FPAA), its decomposed circuit is in $2n$-qubit register.


In [2]:
from qruntest.fpaa_circuit import fpaa_in_circuit
from qruntest.utils import keep
from qruntest.symbolic_cost_model import sym_static_moment_cost_one_grover_iterate
from sympy import symbols
import math,cirq

num_qbit = symbols("n")
pgm_cost = symbols("x") 
depth_cost_fmla = sym_static_moment_cost_one_grover_iterate()
print(f"Depth of Single Grover Operator : {depth_cost_fmla} (x for target program cost )")
for n in [4,6]: # simulation for checking unitary equivalence on >=6 qubits raises significant running time expense (or memory out).
    print(f"Target Program Qubit Len : {n}")
    targ_pgm_qbits = cirq.LineQubit.range(n)
    temp_hadamards = cirq.Circuit([cirq.H(x) for x in targ_pgm_qbits]) # assuming target program depth cost to be 1
    fpaa_one_grover_iterate = fpaa_in_circuit(pgm_qc = temp_hadamards,
                                                l = 1,
                                                delta = math.sqrt(0.05),
                                                targ_reduce=True)
    decomposed = cirq.Circuit( [x for x in cirq.decompose(fpaa_one_grover_iterate, keep=keep)])
    fmal_res = depth_cost_fmla.subs({pgm_cost : 1, num_qbit:n})
    assert len(decomposed) == fmal_res 
    assert cirq.allclose_up_to_global_phase(cirq.unitary(fpaa_one_grover_iterate), cirq.unitary(decomposed)) # to skip the simulation, comment here
    print(fpaa_one_grover_iterate)
    print("Assertion Passed")

Depth of Single Grover Operator : 42*n + 2*x - 52
Target Program Qubit Len : 4
                                                                    ┌──────────────┐                           ┌────────────┐
0: ───(0)───────────────────────────────────────────────(0)───PGM───────────────(0)───────────────────────────────────────────────────────────────(0)───PGM───
      │                                                 │     │                 │                                                                 │     │
1: ───(0)───────────────────────────────────────────────(0)───PGM───────────────(0)───────────────────────────────────────────────────────────────(0)───PGM───
      │                                                 │     │                 │                                                                 │     │
2: ───┼─────(0)───────────────────────────────────(0)───┼─────PGM───────────────┼──────(0)──────────────────────────────────────────────────(0)───┼─────PGM───
      │   